# Concurrency

Async kernel runs [message request](https://jupyter-client.readthedocs.io/en/stable/messaging.html#messaging-in-jupyter) with one of the following `RunModes`.

- blocking: Run the handler directly blocking the message loop
- queue: Run in a queue
- task: Run in a task
- thread: Run in a Caller thread

The kernel decides the run mode dynamically with the method [get_run_mode][async_kernel.kernel.Kernel.get_run_mode].

In [ ]:
from async_kernel import utils
from async_kernel.typing import KernelConcurrencyMode, MsgType, RunMode

kernel = utils.get_kernel()

kernel.get_run_mode(MsgType.comm_msg)

The `kernel.concurrency_mode` will also change the `RunMode` that is returned by `kernel.get_run_mode`.

In [ ]:
kernel.concurrency_mode = KernelConcurrencyMode.blocking
print(f"""
kernel.concurrency_mode: {kernel.concurrency_mode!s}    
kernel.get_run_mode:     {kernel.get_run_mode(MsgType.comm_msg).name}""")

kernel.concurrency_mode = KernelConcurrencyMode.default
print(f"""
kernel.concurrency_mode: {kernel.concurrency_mode!s}    
kernel.get_run_mode:     {kernel.get_run_mode(MsgType.comm_msg).name}""")

Below is a list of the run modes for the currently available concurrency modes.

<div class="admonition note">
    <p class="admonition-title">Note</p>
    <p>
    `blocking` mode is roughly equivalent to how IpyKernel < 7.0 operates. 
    </p>
</div>


In [ ]:
data = kernel.all_concurrency_run_modes()
try:
    import pandas as pd
except ImportError:
    print(data)
else:
    data = pd.DataFrame(data)
    data["RunMode"] = data.RunMode.str.replace("##", "")
    data = data.pivot(index="MsgType", columns=["SocketID", "KernelConcurrencyMode"], values="RunMode")  # noqa: PD010
    display(data)

## Execute request run mode

There are a few options to modify how code cells are run.

- Metadata
- Directly in code
- tags
- Message header (in custom messages)


<div class="admonition warning">
    <p class="admonition-title">Warning</p>
    <p>Only Jupyter lab is known to allow concurrent execution of cells.</p>
</div>


### Code for example

- **This example requires ipywidgets**
- **Ensure you are running an async kernel**

Lets define a function that we'll reuse for the remainder of the notebook.

In [ ]:
async def demo():
    import threading

    from aiologic import Event
    from ipywidgets import Button

    from async_kernel import utils

    print("Run mode:", utils.get_job()["run_mode"])

    print(f"Thread name: '{threading.current_thread().name}'")
    button = Button(description="Finish")
    event = Event()
    button.on_click(lambda _: event.set())
    display(button)
    await event
    button.close()
    print(f"Finished ... thread name: '{threading.current_thread().name}'")
    return "Finished"

Lets run it normally (queue)

In [ ]:
await demo()

In [ ]:
##queue
# Tip: try running this cell while the previous cell is still busy.
await demo()

### Run mode: task

The `task` mode instructs the kernel to execute the code in a task separate to the queue, Both `task` and `thread` execute modes can be started when the kernel is *busy executing*. There is no imposed limitation on the number of tasks (or threads) that can be run concurrently.

See also the [Caller](caller.ipynb#caller) example on how to call directly.

In [ ]:
RunMode.task  # noqa: B018  # Using the literal `RunMode` values directly is also possible. Though it may show up as a [Flake8 B018 issue](https://docs.astral.sh/ruff/rules/useless-expression/)
await demo()

### Run mode: thread

In [ ]:
# This time we'll use the tag to run the cell in a Thread
await demo()

In [ ]:
##thread
%callers # magic provided by async kernel